In [41]:
import re
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
webpage = "https://www.anvur.it/attivita/classificazione-delle-riviste/classificazione-delle-riviste-ai-fini-dellabilitazione-scientifica-nazionale/elenchi-di-riviste-scientifiche-e-di-classe-a/"

In [42]:
headers = {
    'User-Agent': 'Anvur Journal Acquisition',
    "Mail": "luigi.palumbo@unitus.it"
}

In [8]:
with requests.get(webpage, headers=headers) as res:
    response = BeautifulSoup(res.text, "html.parser")

In [35]:
link_list = [
    {
        "area": item.get_text(strip=True),
        "filename": item.get("href").split("/")[-1],
        "link": item.get("href")
    }
    for item in
    response.find_all("a", {"href": re.compile("pdf$")})
]

In [36]:
link_classe_a = [
    item for item
    in link_list
    if item.get("area").endswith("classe A")
]

In [37]:
link_scientifici = [
    item for item
    in link_list
    if item not in link_classe_a
]

In [38]:
link_df = pd.DataFrame(link_list)

In [39]:
link_df.to_csv("data/anvur_journals.csv", index=False)

In [48]:
for item in link_list:
    time.sleep(2)
    with requests.get(item.get("link"), headers=headers) as res:
        with open("data/{}".format(item.get("filename")), "wb") as f:
            f.write(res.content)